In [1]:
import numpy as np
import duckdb
import pandas as pd
import seaborn as sns
import math
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score

In [2]:
data = pd.read_csv("BDC_2024_Womens_Data.csv")

# Aggregated Goal Rate of Success Per Player

In [8]:
# Divide based on team one canda and one usa
# Corsi For (CF) = Shot attempts for at even strength: Shots + Blocks + Misses[2]
# Corsi Against (CA) = Shot attempts against at even strength: Shots + Blocks + Misses
shot_attempts = duckdb.sql("""
                            SELECT Player, Team,
                            COUNT(Event) AS "Total Attempted",
                            COUNT (CASE WHEN Event = 'Goal' THEN 1 END) AS "Success"
                            FROM data
                            WHERE Event = 'Goal' OR Event = 'Shot'
                            GROUP BY Player, Team
                           """).df()

shot_attempts["Success Rate"] = round(
    shot_attempts["Success"]/shot_attempts["Total Attempted"], 4)

In [9]:
shot_attempts

,Player,Team,Total Attempted,Success,Success Rate
0,Hilary Knight,Women - United States,16,2,0.1250
1,Brianne Jenner,Women - Canada,21,1,0.0476
2,Taylor Heise,Women - United States,16,1,0.0625
3,Alex Carpenter,Women - United States,15,3,0.2000
4,Megan Keller,Women - United States,14,1,0.0714
5,Grace Zumwinkle,Women - United States,3,0,0.0000
6,Marie-Philip Poulin,Women - Canada,16,0,0.0000
7,Renata Fast,Women - Canada,20,0,0.0000
8,Jocelyne Larocque,Women - Canada,16,0,0.0000
9,Hannah Bilka,Women - United States,12,1,0.0833


# Aggregated Pass Success Rates Per Player

In [10]:
play_counts = duckdb.sql("""
                            SELECT Player, Team,
                            COUNT(Event) AS "Total",
                            COUNT (CASE WHEN Event = 'Play' THEN 1 END) AS "Play",
                            COUNT (CASE WHEN Event = 'Incomplete Play' THEN 1 END) AS "IncPlay"
                            FROM data
                            WHERE Event = 'Play' OR Event = 'Incomplete Play'
                            GROUP BY Player, Team
                           """).df()

play2_counts = duckdb.sql("""
                            SELECT "Player 2" AS Player2, Team,
                            COUNT(Event) AS "Total",
                            COUNT (CASE WHEN Event = 'Play' THEN 1 END) AS "Play",
                            COUNT (CASE WHEN Event = 'Incomplete Play' THEN 1 END) AS "IncPlay"
                            FROM data
                            WHERE Event = 'Play' OR Event = 'Incomplete Play'
                            GROUP BY "Player 2", Team
                           """).df()

joint_play_count = pd.concat([play_counts, play2_counts], axis = 0)

joint_play_count['Player'] = joint_play_count['Player'].fillna(joint_play_count['Player2'])
joint_play_count = joint_play_count.drop(columns=['Player2'])

joint_play_count = duckdb.sql("""
                              SELECT Player, Team,
                              SUM(Total) AS Total,
                              SUM (Play) AS Play,
                              SUM(IncPlay) AS IncPlay
                              FROM joint_play_count
                              GROUP BY Player, Team
                              """).df()


joint_play_count["Rate"] = joint_play_count["Play"] / joint_play_count["Total"]
#print (joint_play_count)

lowest = joint_play_count.nsmallest(5, "Rate", 'all')
print (lowest)

                Player                   Team  Total  Play  IncPlay      Rate
53       Casey O'Brien  Women - United States    7.0   3.0      4.0  0.428571
14  Natalie Buchbinder  Women - United States   19.0  10.0      9.0  0.526316
21     Hayley Scamurra  Women - United States   80.0  50.0     30.0  0.625000
20   Jamie Lee Rattray         Women - Canada   60.0  38.0     22.0  0.633333
17         Emily Clark         Women - Canada  127.0  81.0     46.0  0.637795


# Aggregated Faceoff Win Rate Per Player

In [13]:
faceoff_win = duckdb.sql("""
                    SELECT Player, Team, 
                    COUNT (Event) AS "Faceoff Wins"
                    FROM data
                    WHERE (Event = 'Faceoff Win')
                    GROUP BY Player, Team
                    """).df()

faceoff_lost = duckdb.sql("""
                          SELECT "Player 2", Team,
                          COUNT (Event) AS "Faceoff Loses"
                          FROM data
                          WHERE (Event = 'Faceoff Win')
                          GROUP BY Player, Team
                          """)

joint_faceoff_count = pd.concat([play_counts, play2_counts], axis = 0)

joint_play_count['Player'] = joint_play_count['Player'].fillna(joint_play_count['Player2'])
joint_play_count = joint_play_count.drop(columns=['Player2'])

joint_play_count = duckdb.sql("""
                              SELECT Player, Team,
                              SUM(Total) AS Total,
                              SUM (Play) AS Play,
                              SUM(IncPlay) AS IncPlay
                              FROM joint_play_count
                              GROUP BY Player, Team
                              """).df()


joint_play_count["Rate"] = joint_play_count["Play"] / joint_play_count["Total"]

In [14]:
faceoff_win

,Player,Team,Faceoff Wins
0,Marie-Philip Poulin,Women - Canada,42
1,Kelly Pannek,Women - United States,20
2,Taylor Heise,Women - United States,33
3,Alex Carpenter,Women - United States,24
4,Kristin O'Neill,Women - Canada,25
5,Abby Roque,Women - United States,32
6,Julia Gosling,Women - Canada,6
7,Brianne Jenner,Women - Canada,2
8,Lacey Eden,Women - United States,1
9,Sarah Fillier,Women - Canada,3


In [19]:
a = data["Away Team Skaters"].unique()
h = data["Home Team Skaters"].unique()

print (a)
print (h)

away = duckdb.sql("""
                              SELECT "Away Team Skaters", "Home Team Skaters", Date
                              FROM data
                              GROUP BY Date, "Away Team Skaters", "Home Team Skaters"
                              """).df()

print (away)

# does being a power play situation 
# does more man power increase success of shot attempted? 

[5 4 3]
[5 4 3]
    Away Team Skaters  Home Team Skaters        Date
0                   5                  5  2023-11-08
1                   4                  5  2023-11-08
2                   5                  4  2023-11-08
3                   4                  4  2023-11-08
4                   5                  5  2023-11-11
5                   5                  4  2023-11-11
6                   4                  5  2023-11-11
7                   3                  5  2023-11-11
8                   4                  4  2023-11-11
9                   5                  5  2023-12-14
10                  4                  5  2023-12-14
11                  5                  4  2023-12-14
12                  4                  3  2023-12-14
13                  5                  5  2023-12-16
14                  5                  4  2023-12-16
15                  4                  5  2023-12-16
16                  3                  5  2023-12-16
17                  3         